In [7]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np
from math import ceil

In [6]:
path_to_data = 'C:/Users/PetukhovMD/Desktop/szp_2022/'
path_to_docs = 'C:/Users/PetukhovMD/Desktop/справочники/'
months = ['feb', 'jan', 'mar']
sheet = 'Sheet'

In [9]:
def to_double(row, f_name):
    if type(row[f_name]) is float:
        return row[f_name] 
    t = str(row[f_name]).split(',')
    if t[0] == 'nan':
        return 0.0
    return float(t[0]+'.'+t[1])

In [33]:
def load_data(file):
    df = pd.read_excel(path_to_data + file + '.xlsx')
    df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
    sample = df[(df.type == 'Основное место работы') | (df.type == 'Внутреннее совместительство')]
    sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index().groupby('snils').max().reset_index()
    sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
    sums = sums[(sums.stv >= 1.0) & (sums.status == 'Работа') & (sums.day == 1)]
    sums = sums[['inn_y', 'snils', 'sum_x', 'stv', 'job']].drop_duplicates(['inn_y', 'snils', 'sum_x'])
    sums = sums.rename(columns = {'inn_y': 'inn', 'sum_x': 'sum_' + file, 'job':'job_' + file, 'stv': 'stv_' + file})
    return sums

In [34]:
def create_res(months):
    res = 0
    for i in range(len(months)):
        if i == 0:
            res = load_data(months[i])
        else:
            res = pd.merge(res, load_data(months[i]), how='outer', on=['inn', 'snils'])
    return res

In [24]:
file = 'mar'
d = pd.read_excel(path_to_data + file + '.xlsx')

In [25]:
d.stv = d.apply(lambda row: to_double(row, 'stv'), axis=1)

In [26]:
s = d[~(d.type == 'Внешнее совместительство')]
ss = s.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index()

In [28]:
ss = pd.merge(ss, s[s.type == 'Основное место работы'], how='left', on=['inn', 'snils'])

In [53]:
ss[ss.job.isna()]

,inn,snils,sum_x,sum_y,job,day,type,stv,status,status_pref
4848,5051005670,013-184-668 28,9333.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4865,5051005670,014-688-674 79,23100.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5164,5051005670,073-954-680 04,21000.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5287,5051005670,117-332-744 39,9333.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6615,5074045703,038-825-734 88,10400.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
165376,9715206976,156-020-820 28,6000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165405,9715206976,166-110-242 25,46363.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165425,9715206976,171-990-672 03,26250.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165434,9715206976,174-534-080 69,21050.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
ss.dropna(how='all', subset=['job'])[ss.sum_x != ss.sum_y]

C:\Users\PetukhovMD\AppData\Local\Temp\ipykernel_23168\1276137153.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ss.dropna(how='all', subset=['job'])[ss.sum_x != ss.sum_y]


,inn,snils,sum_x,sum_y,job,day,type,stv,status,status_pref
718,5003021368,154-314-049 32,0.00,NaN,Учитель,0.0,Основное место работы,1.00,Увольнение,Увольнение
975,5003021495,001-703-734 92,59925.00,52663.00,Помощник воспитателя,1.0,Основное место работы,1.00,Работа,Работа
1011,5003021495,010-374-085 98,75003.80,64110.80,Уборщик служебных и производственных помещений,1.0,Основное место работы,1.00,Работа,Работа
1017,5003021495,011-347-332 98,71819.97,58860.72,Музыкальный руководитель,1.0,Основное место работы,1.00,Работа,Работа
1029,5003021495,014-158-355 22,60830.09,53568.09,Рабочий по комплексному обслуживанию и ремонту...,1.0,Основное место работы,1.00,Работа,Работа
...,...,...,...,...,...,...,...,...,...,...
165487,9715206976,204-248-388 42,115000.00,85500.00,Учитель-логопед,1.0,Основное место работы,1.00,Работа,Работа
165488,9715206976,204-279-880 68,116937.50,96550.00,Специалист по сетевому администрированию,1.0,Основное место работы,1.00,Работа,Работа
165494,9715206976,210-512-230 81,130065.11,96415.11,Учитель,1.0,Основное место работы,1.05,Работа,Работа
165495,9715206976,212-203-372 89,24816.08,24066.08,Педагог дополнительного образования,0.0,Основное место работы,1.00,Отпуск неоплачиваемый по разрешению работодателя,Работа


In [55]:
ss

,inn,snils,sum_x,sum_y,job,day,type,stv,status,status_pref
0,5003021368,001-171-362 75,61500.00,61500.00,Помощник воспитателя,1.0,Основное место работы,1.0,Работа,Работа
1,5003021368,001-191-281 85,82600.36,82600.36,Учитель,0.0,Основное место работы,1.0,Работа,Работа
2,5003021368,001-244-107 65,109077.82,109077.82,Учитель,1.0,Основное место работы,1.0,Работа,Работа
3,5003021368,001-263-234 77,87744.00,87744.00,Воспитатель,1.0,Основное место работы,1.0,Работа,Работа
4,5003021368,001-322-594 80,81798.18,81798.18,Специалист,1.0,Основное место работы,1.0,Работа,Работа
...,...,...,...,...,...,...,...,...,...,...
166739,9718071371,196-882-391 45,77596.00,77596.00,Бухгалтер,1.0,Основное место работы,1.0,Работа,Работа
166740,9718071371,198-036-430 93,110851.00,110851.00,Бухгалтер,1.0,Основное место работы,1.0,Работа,Работа
166741,9718071371,199-418-719 35,145851.00,145851.00,Бухгалтер,1.0,Основное место работы,1.0,Работа,Работа
166742,9718071371,205-412-630 13,189010.00,189010.00,Начальник группы,1.0,Основное место работы,1.0,Работа,Работа


In [4]:
def load_data_all(file):
    df = pd.read_excel(path_to_data + file + '.xlsx')
    df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
    sample = df[~(df.type == 'Внешнее совместительство')]
    sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index().groupby('snils').max().reset_index()
    sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
    sums = sums[['inn_y', 'snils', 'sum_x', 'stv', 'job']].drop_duplicates(['inn_y', 'snils', 'sum_x'])
    sums = sums.rename(columns = {'inn_y': 'inn', 'sum_x': 'sum_' + file, 'job':'job_' + file, 'stv': 'stv_' + file})
    return sums

In [3]:
def create_full_res(months):
    res = 0
    for i in range(len(months)):
        if i == 0:
            res = load_data_all(months[i])
        else:
            res = pd.merge(res, load_data_all(months[i]), how='outer', on=['inn', 'snils'])
    return res

In [80]:
res = create_res(months)

In [81]:
res[res.job_mar.isna() == False]

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar
1,7.736615e+09,001-010-312 25,146547.14,1.0,Учитель,164214.30,1.0,Учитель,160599.30,1.0,Учитель
2,7.723170e+09,001-011-996 67,167905.34,1.0,Учитель,161441.63,1.0,Учитель,166752.15,1.0,Учитель
3,7.727124e+09,001-017-453 65,87135.84,1.0,Секретарь руководителя,90135.84,1.0,Документовед,86627.92,1.0,Секретарь руководителя
4,7.729776e+09,001-020-711 41,60020.00,1.0,Воспитатель,65993.00,1.0,Воспитатель,68000.00,1.0,Воспитатель
5,7.723356e+09,001-021-199 51,61918.39,1.0,Преподаватель,66502.39,1.0,Преподаватель,59040.96,1.0,Преподаватель
...,...,...,...,...,...,...,...,...,...,...,...
153540,7.734409e+09,215-089-180 55,NaN,NaN,NaN,NaN,NaN,NaN,25039.34,1.0,Специалист
153541,7.736615e+09,215-093-175 39,NaN,NaN,NaN,NaN,NaN,NaN,38178.00,1.0,Уборщик территории
153542,7.743021e+09,215-094-406 39,NaN,NaN,NaN,NaN,NaN,NaN,10015.73,1.0,Рабочий по комплексному обслуживанию и ремонту...
153543,7.736051e+09,215-120-882 18,NaN,NaN,NaN,NaN,NaN,NaN,13916.40,1.0,Рабочий по комплексному обслуживанию и ремонту...


In [10]:
def print_df(df, name):
    wb = Workbook()
    ws = wb.active
    for  i in df_to_row(df, header=True):
        ws.append(i)
    wb.save(name + '.xlsx')

In [142]:
print_df(res[(res.inn == 5003021368) & (res.job_feb == 'Учитель')], 'teach_check')

In [287]:
res[(res.inn == 9718071371)]

,inn,snils,sum_jan,job_jan,sum_feb,job_feb
112515,9718071371,001-102-623 46,147930.0,ведущий Бухгалтер,157930.00,ведущий Бухгалтер
112516,9718071371,001-184-402 83,177930.0,ведущий Бухгалтер,177930.00,ведущий Бухгалтер
112517,9718071371,001-200-216 33,115851.0,Бухгалтер,142896.32,Бухгалтер
112518,9718071371,001-208-598 92,96078.0,Бухгалтер,117705.00,Бухгалтер
112519,9718071371,001-257-058 90,114851.0,Бухгалтер,110851.00,Бухгалтер
...,...,...,...,...,...,...
131871,9718071371,166-448-762 08,NaN,NaN,110851.00,Бухгалтер
131872,9718071371,173-386-417 88,NaN,NaN,101079.63,Бухгалтер
131873,9718071371,175-876-580 36,NaN,NaN,110851.00,Бухгалтер
131874,9718071371,176-772-921 22,NaN,NaN,110851.00,Инженер-сметчик


In [11]:
def szp(row, months):
    if pd.isna(row['job_' + months[-1]]):
        return np.nan
    cnt = 0
    sum_szp = 0
    for month in months:
        if pd.isna(row['sum_' + month]):
            continue
        cnt += 1
        sum_szp += row['sum_' + month]
    # if row['inn'] == 9715217689 and row['snils'] == '173-260-871 66':
    #     print(cnt, sum_szp)
    return sum_szp / cnt

In [36]:
res['szp'] = res.apply(lambda row: szp(row, months), axis= 1)

In [55]:
depr_month_inc = 84100.0

In [16]:
df = pd.read_excel(path_to_docs + 'input.xlsx', sheet_name='УГД')

In [17]:
df

,Должность,УГД,Педагогический,Работники непосредственно осуществляющие и обеспечивающие основной учебно-вспомогательный процесс во взаимодействии с детьми
0,Главный бухгалтер,Административно-управленческий персонал,0,0
1,Директор,Административно-управленческий персонал,0,0
2,Директор филиала,Административно-управленческий персонал,0,0
3,Заведующий,Административно-управленческий персонал,0,0
4,Заместитель главного бухгалтера,Административно-управленческий персонал,0,0
...,...,...,...,...
291,Эксперт,Прочий персонал (включая иной административный...,0,0
292,Электромеханик,Прочий персонал (включая иной административный...,0,0
293,Старший юрисконсульт,Прочий персонал (включая иной административный...,0,0
294,Ведущий юрисконсульт,Прочий персонал (включая иной административный...,0,0


In [18]:
ped = df[df['Педагогический'] == 1]['Должность'].tolist()
ped

['Ассистент',
 'Декан факультета',
 'Директор института',
 'Доцент',
 'Заведующий кафедрой',
 'Профессор',
 'Преподаватель',
 'Старший преподаватель',
 'Воспитатель',
 'Инструктор-методист',
 'Инструктор по труду',
 'Инструктор по физической культуре',
 'Концертмейстер',
 'Мастер производственного обучения',
 'Методист',
 'Музыкальный руководитель',
 'Педагог дополнительного образования',
 'Педагог-библиотекарь',
 'Педагог-организатор',
 'Педагог-психолог ',
 'Преподаватель ',
 'Преподаватель-организатор основ безопасности жизнедеятельности',
 'Руководитель физического воспитания',
 'Социальный педагог',
 'Старший вожатый',
 'Старший воспитатель',
 'Старший инструктор-методист',
 'Старший методист',
 'Старший педагог дополнительного образования',
 'Старший педагог-психолог',
 'Старший тренер-преподаватель',
 'Тренер-преподаватель',
 'Тьютор',
 'Учитель',
 'Учитель-дефектолог',
 'Учитель-логопед']

In [19]:
isp = df[df['Работники непосредственно осуществляющие и обеспечивающие основной учебно-вспомогательный процесс во взаимодействии с детьми'] == 1]['Должность'].tolist()
isp

['Преподаватель',
 'Воспитатель',
 'Инструктор по физической культуре',
 'Концертмейстер',
 'Мастер производственного обучения',
 'Музыкальный руководитель',
 'Педагог дополнительного образования',
 'Педагог-организатор',
 'Педагог-психолог ',
 'Преподаватель ',
 'Социальный педагог',
 'Тьютор',
 'Учитель',
 'Учитель-дефектолог',
 'Учитель-логопед',
 'Помощник воспитателя',
 'Младший воспитатель']

In [44]:
vosp = ['Старший воспитатель', 'Воспитатель']

In [37]:
def szp_ped(row, months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in ped:
            cnt += 1
            sum_szp += row['sum_' + month]
    if cnt == 0 or pd.isna(row['job_' + months[-1]]):
        return np.nan
    return sum_szp / cnt

In [13]:
def szp_teach(row, months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in ['Учитель']:
            cnt += 1
            sum_szp += row['sum_' + month]
    if cnt == 0 or row['job_' + months[-1]] != 'Учитель':
        return np.nan
    return sum_szp / cnt

In [45]:
def szp_vosp(row, months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in vosp:
            cnt += 1
            sum_szp += row['sum_' + month]
    if cnt == 0 or row['job_' + months[-1]] not in vosp:
        return np.nan
    return sum_szp / cnt

In [38]:
res['szp_ped'] = res.apply(lambda row: szp_ped(row, months), axis=1)

In [39]:
res['szp_teach'] = res.apply(lambda row: szp_teach(row, months), axis=1)

In [40]:
res['szp_vosp'] = res.apply(lambda row: szp_vosp(row, months), axis=1)

In [41]:
buckets_teach_old = [(0, 80), (80, 90), (90, 100), (100, 110), (110, 120), (120, 130), (130, 140), (140, 150), (150, 1000)]
buckets_teach_new = [(0, 90), (90, 100), (100, 110), (110, 120), (120, 130), (130, 140), (140, 150), (150, 1000)]
buckets_vosp_old = [(0, 50), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100), (100, 110), (110, 120), (120, 130), (130, 140), (140, 150), (150, 1000)]
buckets_vosp_new = [(0, 60), (60, 80), (80, 100), (100, 110), (110, 120), (120, 130), (130, 140), (140, 150), (150, 1000)]
teach_szp_levels_old = {}
teach_szp_levels_new = {}
vosp_szp_levels_old = {}
vosp_szp_levels_new = {}

In [103]:
print_df(temp, 'temp')

In [104]:
temp

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar,szp_vosp,szp_teach
0,7.720259e+09,001-005-586 64,124968.88,1.0,Заведующий хозяйством,140968.88,1.0,Заведующий хозяйством,60434.74,1.0,Заведующий хозяйством,NaN,NaN
1,7.718310e+09,001-005-786 70,61833.04,1.0,Учитель,55448.10,1.0,Учитель,85809.12,1.0,Учитель,NaN,67696.753333
2,7.735601e+09,001-008-669 78,66928.99,1.0,Учитель,53228.87,1.0,Учитель,75124.20,1.0,Учитель,NaN,65094.020000
3,7.736615e+09,001-010-312 25,146547.14,1.0,Учитель,164214.30,1.0,Учитель,160599.30,1.0,Учитель,NaN,157120.246667
4,7.723170e+09,001-011-996 67,167905.34,1.0,Учитель,161441.63,1.0,Учитель,166752.15,1.0,Учитель,NaN,165366.373333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
176839,NaN,215-089-180 55,NaN,NaN,NaN,NaN,NaN,NaN,25039.34,NaN,NaN,NaN,NaN
176840,NaN,215-093-175 39,NaN,NaN,NaN,NaN,NaN,NaN,38178.00,NaN,NaN,NaN,NaN
176841,NaN,215-094-406 39,NaN,NaN,NaN,NaN,NaN,NaN,10015.73,NaN,NaN,NaN,NaN
176842,NaN,215-120-882 18,NaN,NaN,NaN,NaN,NaN,NaN,13916.40,NaN,NaN,NaN,NaN


In [33]:
res[res.szp_teach > 150000]

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar,szp,szp_ped,szp_teach,szp_vosp
1,7736614860,001-010-312 25,146547.14,1.00,Учитель,164214.30,1.0,Учитель,160599.30,1.00,Учитель,157120.246667,157120.246667,157120.246667,NaN
2,7723169690,001-011-996 67,167905.34,1.00,Учитель,161441.63,1.0,Учитель,166752.15,1.00,Учитель,165366.373333,165366.373333,165366.373333,NaN
13,7719825934,001-025-260 55,156242.71,1.55,Учитель,NaN,NaN,NaN,158344.35,1.55,Учитель,157293.530000,157293.530000,157293.530000,NaN
79,7702833359,001-131-908 67,160320.62,1.00,Учитель,152088.02,1.0,Учитель,175707.24,1.00,Учитель,162705.293333,162705.293333,162705.293333,NaN
150,7727190251,001-170-050 58,189029.36,1.00,Учитель,135648.26,1.0,Учитель,160115.58,1.00,Учитель,161597.733333,161597.733333,161597.733333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152369,7732117313,202-553-103 04,NaN,NaN,NaN,NaN,NaN,NaN,176682.63,1.00,Учитель,176682.630000,176682.630000,176682.630000,NaN
152397,7734687492,204-064-987 41,NaN,NaN,NaN,NaN,NaN,NaN,185115.44,1.00,Учитель,185115.440000,185115.440000,185115.440000,NaN
152451,7729783137,209-661-850 84,NaN,NaN,NaN,NaN,NaN,NaN,157003.58,1.00,Учитель,157003.580000,157003.580000,157003.580000,NaN
152453,7731582090,209-723-071 59,NaN,NaN,NaN,NaN,NaN,NaN,167283.07,1.00,Учитель,167283.070000,167283.070000,167283.070000,NaN


In [42]:
for bucket in buckets_teach_old:
    teach_szp_levels_old[str(bucket[0]) + '-' + str(bucket[1])] = temp[temp['szp_teach'].between(bucket[0] * 1000, bucket[1] * 1000, inclusive='left')]['snils'].count()
teach_szp_levels_old

{'0-80': 3540,
 '80-90': 4892,
 '90-100': 6734,
 '100-110': 7490,
 '110-120': 7942,
 '120-130': 7497,
 '130-140': 6227,
 '140-150': 4678,
 '150-1000': 7946}

In [57]:
for bucket in buckets_teach_new:
    teach_szp_levels_new[str(bucket[0]) + '-' + str(bucket[1])] = temp[temp['szp_teach'].between(bucket[0] * 1000, bucket[1] * 1000, inclusive='left')]['snils'].count()
teach_szp_levels_new

{'0-90': 8315,
 '90-100': 6703,
 '100-110': 7480,
 '110-120': 7932,
 '120-130': 7496,
 '130-140': 6213,
 '140-150': 4678,
 '150-1000': 7918}

In [46]:
for bucket in buckets_vosp_old:
    vosp_szp_levels_old[str(bucket[0]) + '-' + str(bucket[1])] = temp[temp['szp_vosp'].between(bucket[0] * 1000, bucket[1] * 1000, inclusive='left')]['snils'].count()
vosp_szp_levels_old

{'0-50': 2369,
 '50-60': 5050,
 '60-70': 7433,
 '70-80': 6601,
 '80-90': 4175,
 '90-100': 2516,
 '100-110': 1448,
 '110-120': 826,
 '120-130': 500,
 '130-140': 295,
 '140-150': 144,
 '150-1000': 260}

In [59]:
for bucket in buckets_vosp_new:
    vosp_szp_levels_new[str(bucket[0]) + '-' + str(bucket[1])] = temp[temp['szp_vosp'].between(bucket[0] * 1000, bucket[1] * 1000, inclusive='left')]['snils'].count()
vosp_szp_levels_new

{'0-60': 7339,
 '60-80': 14018,
 '80-100': 6691,
 '100-110': 1445,
 '110-120': 829,
 '120-130': 500,
 '130-140': 296,
 '140-150': 144,
 '150-1000': 260}

In [78]:
print_df(temp[temp.szp_vosp.isna() == False][['inn', 'snils', 'stv_jan', 'stv_feb', 'stv_mar', 'job_mar', 'szp_vosp']], 'Воспитатели')

In [47]:
temp

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar,szp_vosp,szp_teach
0,7.720259e+09,001-005-586 64,124968.88,1.0,Заведующий хозяйством,140968.88,1.0,Заведующий хозяйством,60434.74,1.0,Заведующий хозяйством,NaN,NaN
1,7.718310e+09,001-005-786 70,61833.04,1.0,Учитель,55448.10,1.0,Учитель,85809.12,1.0,Учитель,NaN,67696.753333
2,7.735601e+09,001-008-669 78,66928.99,1.0,Учитель,53228.87,1.0,Учитель,75124.20,1.0,Учитель,NaN,65094.020000
3,7.736615e+09,001-010-312 25,146547.14,1.0,Учитель,164214.30,1.0,Учитель,160599.30,1.0,Учитель,NaN,157120.246667
4,7.723170e+09,001-011-996 67,167905.34,1.0,Учитель,161441.63,1.0,Учитель,166752.15,1.0,Учитель,NaN,165366.373333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
176757,7.734409e+09,215-089-180 55,NaN,NaN,NaN,NaN,NaN,NaN,25039.34,1.0,Специалист,NaN,NaN
176758,7.736615e+09,215-093-175 39,NaN,NaN,NaN,NaN,NaN,NaN,38178.00,1.0,Уборщик территории,NaN,NaN
176759,7.743021e+09,215-094-406 39,NaN,NaN,NaN,NaN,NaN,NaN,10015.73,1.0,Рабочий по комплексному обслуживанию и ремонту...,NaN,NaN
176760,7.736051e+09,215-120-882 18,NaN,NaN,NaN,NaN,NaN,NaN,13916.40,1.0,Рабочий по комплексному обслуживанию и ремонту...,NaN,NaN


In [365]:
res[res.snils == '047-832-306 66']

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar,szp_ped,szp_teach,szp
136289,9718071371,047-832-306 66,NaN,NaN,NaN,469642.0,1.0,Первый заместитель начальника,494642.0,1.0,Первый заместитель начальника,NaN,NaN,482142.0


In [487]:
val_5 = res[['inn', 'szp_ped']].groupby('inn').mean()
val_5['value'] = val_5['szp_ped'] / depr_month_inc
val_5

,szp_ped,value
inn,,
5003021368,102867.349708,1.223155
5003021495,111682.021761,1.327967
5003021640,111310.630572,1.323551
5003096290,103696.278993,1.233012
5030032182,108526.287245,1.290443
...,...,...
7743940590,129138.584406,1.535536
9705101759,NaN,NaN
9715206976,115953.025083,1.378752


In [488]:
print_df(val_5, 'val_5')
print_df(res[res.szp_ped.isna() == False][['inn', 'snils', 'szp_ped']], 'val_5_ppl')

In [489]:
val_6 = res[['inn', 'szp_ped']].groupby('inn').quantile(0.5).rename(columns={'szp_ped': 'q'})
val_6 = pd.merge(val_6, res[['inn', 'snils', 'szp_ped']], how='right', on='inn')
val_6_out = val_6
val_6 = val_6[val_6.szp_ped <= val_6.q][['inn', 'szp_ped']].groupby('inn').mean()
val_6

,szp_ped
inn,
5003021368,76511.005960
5003021495,85499.372325
5003021640,80420.183333
5003096290,74238.593684
5030032182,75085.680529
...,...
7743937357,97593.493535
7743939315,75548.254524
7743940590,102726.402673


In [490]:
print_df(val_6, 'val_6')
print_df(val_6_out, 'val_6_ppl')

In [370]:
res[res.snils == '154-824-084 70']

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar,szp_ped,szp_teach,szp
22433,7715310719,154-824-084 70,95227.31,1.0,Учитель,95508.44,1.0,Учитель,94743.63,1.0,Учитель,95159.793333,95159.793333,95159.793333


In [491]:
val_7 = pd.merge(res[['inn', 'szp_teach']].groupby('inn').count().rename(columns={'szp_teach': 'cnt1'}), 
res[res.szp_teach >= 84100][['inn', 'szp_teach']].groupby('inn').count().rename(columns={'szp_teach': 'cnt2'}), how='outer', on='inn')
val_7['val'] = val_7.cnt2 / val_7.cnt1
val_7[val_7.val < 1]

,cnt1,cnt2,val
inn,,,
5003021368,301,298.0,0.990033
5030032182,66,65.0,0.984848
5074019220,103,102.0,0.990291
5074045703,180,178.0,0.988889
7701377657,60,59.0,0.983333
...,...,...,...
7736076587,199,198.0,0.994975
7743020747,34,31.0,0.911765
7743021300,55,54.0,0.981818


In [492]:
print_df(val_7, 'val_7')
print_df(res[res.szp_teach.isna() == False][['inn', 'snils', 'szp_teach']], 'val_7_ppl')

In [206]:
def q_cnt1(row):
    if pd.isna(row['szp']):
        return np.nan
    return row['szp'] if row['szp'] <= row['0.1'] else np.nan

In [207]:
def q_cnt2(row):
    if pd.isna(row['szp']):
        return np.nan
    return row['szp'] if row['szp'] >= row['0.9'] else np.nan

In [493]:
q01 = res[['inn','szp']].groupby('inn').quantile(0.1).rename(columns={'szp':'0.1'})
q09 = res[['inn','szp']].groupby('inn').quantile(0.9).rename(columns={'szp':'0.9'})
q = pd.merge(q01, q09, how='right', on='inn')
qm = pd.merge(res[['inn', 'szp']], q, how='left', on='inn')
qm['cnt1'] = qm.apply(lambda row: q_cnt1(row), axis=1)
qm['cnt2'] = qm.apply(lambda row: q_cnt2(row), axis=1)
qm_pers = pd.merge(qm[qm['szp'].isna() == False], res[['inn','szp', 'snils']], how='left', on=['inn', 'szp']).drop_duplicates(['inn', 'snils'])
qm = qm[['inn', 'cnt1', 'cnt2']].groupby('inn').mean()
qm['ans'] = qm.cnt2 / qm.cnt1
qm[qm.ans > 3.7]

,cnt1,cnt2,ans
inn,,,
7701293580,78588.095238,294970.708095,3.753376
7702061938,80552.333333,339784.222222,4.218180
7702833359,56040.328200,209469.727533,3.737839
7704040242,84142.640952,372458.901905,4.426518
7704118139,54532.141667,232165.081000,4.257399
...,...,...,...
7736614860,52652.653458,239004.417838,4.539266
7736668110,63611.073889,262844.213056,4.132051
7737066359,67278.787879,262367.497273,3.899706


In [197]:
q01[q01['0.1'].isna()]

,0.1
inn,


In [494]:
print_df(qm, 'dezil')
print_df(qm_pers, 'dezil_pers')

In [495]:
cut_off = res[['inn', 'szp']].groupby('inn').count().rename(columns={'szp': 'cnt'})
cut_off.cnt = np.ceil(cut_off.cnt / 10)
cut_off = cut_off.to_dict()['cnt']
cut_off

{5003021368: 78.0,
 5003021495: 84.0,
 5003021640: 47.0,
 5003096290: 44.0,
 5030032182: 22.0,
 5030032376: 44.0,
 5051005503: 28.0,
 5051005623: 13.0,
 5051005670: 79.0,
 5074019220: 25.0,
 5074019252: 33.0,
 5074045703: 47.0,
 7701032028: 3.0,
 7701050524: 14.0,
 7701113894: 5.0,
 7701293580: 7.0,
 7701375995: 6.0,
 7701377657: 18.0,
 7701384943: 3.0,
 7701406844: 6.0,
 7701905508: 24.0,
 7702058364: 14.0,
 7702061938: 3.0,
 7702764296: 15.0,
 7702797527: 25.0,
 7702833359: 25.0,
 7703021039: 6.0,
 7703112310: 8.0,
 7703261626: 12.0,
 7703363890: 22.0,
 7703379868: 3.0,
 7703611221: 16.0,
 7703738796: 18.0,
 7703742922: 33.0,
 7703776689: 9.0,
 7703808740: 11.0,
 7704040242: 7.0,
 7704118121: 5.0,
 7704118139: 10.0,
 7704178709: 14.0,
 7704191153: 6.0,
 7704222919: 15.0,
 7704222926: 35.0,
 7704231631: 28.0,
 7704234760: 22.0,
 7704271031: 1.0,
 7704853583: 17.0,
 7705020295: 12.0,
 7705041520: 3.0,
 7705052410: 6.0,
 7705399348: 25.0,
 7705480398: 25.0,
 7705513678: 23.0,
 770551373

In [496]:
ppl = res[['inn', 'szp']].sort_values('szp').dropna().groupby('inn')
keys = ppl.groups.keys()
val_1_7 = pd.DataFrame({'inn': [], 'val': []})
val_1_7

,inn,val


In [497]:
for key in keys:
    top = ppl.get_group(key).tail(int(cut_off[key]))['szp'].mean()
    bot = ppl.get_group(key).head(int(cut_off[key]))['szp'].mean()
    t_df = pd.DataFrame({'inn': [key], 'val': [top / bot]})
    val_1_7 = pd.concat([val_1_7, t_df])

In [498]:
print_df(val_1_7, 'val_1_7')

In [499]:
cnt1 = res[['inn', 'szp_teach']].groupby('inn').mean().rename(columns={'szp_teach':'cnt1'})
cnt2 = res[['inn', 'szp_teach']].groupby('inn').quantile(0.5).rename(columns={'szp_teach':'q'})
cnt2 = pd.merge(cnt2, res[['inn', 'snils', 'szp_teach']], how='right', on='inn')
teach_perc_out = cnt2
cnt2 = cnt2[cnt2.szp_teach <= cnt2.q][['inn', 'szp_teach']].groupby('inn').mean().rename(columns={'szp_teach':'cnt2'})
val_15 = pd.merge(cnt2, cnt1, how='left', on='inn')
val_15['val'] = val_15.cnt1 / val_15.cnt2
val_15[val_15.val > 1.3]

,cnt2,cnt1,val
inn,,,
7727213822,128884.032706,171549.964383,1.331041
7733131543,111474.967143,145505.519881,1.305275


In [401]:
def qt01(row):
    if pd.isna(row['szp_teach']):
        return np.nan
    return row['szp_teach'] if row['szp_teach'] <= row['t01'] else np.nan

In [402]:
def qt09(row):
    if pd.isna(row['szp_teach']):
        return np.nan
    return row['szp_teach'] if row['szp_teach'] >= row['t09'] else np.nan

In [404]:
res.szp_teach.describe()

count     41643.000000
mean     127906.620882
std       27607.928482
min        5950.000000
25%      108065.459167
50%      125327.260000
75%      143663.865000
max      353671.556667
Name: szp_teach, dtype: float64

In [464]:
res[res.snils == '150-388-362 61']

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar,szp,szp_ped,szp_teach
95923,7734249770,150-388-362 61,183222.02,1.61,Учитель,203322.02,1.61,Учитель,176122.02,1.61,Учитель,187555.353333,187555.353333,187555.353333


In [500]:
t01 = res[res.szp_teach.isna() == False][['inn', 'szp_teach']].groupby('inn').quantile(0.1).rename(columns={'szp_teach': 't01'})
t09 = res[res.szp_teach.isna() == False][['inn', 'szp_teach']].groupby('inn').quantile(0.9).rename(columns={'szp_teach': 't09'})
qt = pd.merge(t01, t09, how='right', on='inn')
qt = pd.merge(qt, res[res.szp_teach.isna() == False][['inn', 'snils', 'szp_teach']], how='left', on='inn')
qt['qt01'] = qt.apply(lambda row: qt01(row), axis=1)
qt['qt09'] = qt.apply(lambda row: qt09(row), axis=1)
val_2_6 = qt[['inn', 'qt01', 'qt09']].groupby('inn').mean()
val_2_6['dezil'] = val_2_6.qt09 / val_2_6.qt01
val_2_6

,qt01,qt09,dezil
inn,,,
5003021368,91773.760000,170244.768656,1.855048
5003021495,86119.774714,185506.984714,2.154058
5003021640,99426.023590,190845.823205,1.919476
5003096290,93784.508021,192694.359271,2.054650
5030032182,90973.248810,184630.170476,2.029500
...,...,...,...
7743937357,96430.482222,153311.853333,1.589869
7743939315,84414.840000,161996.636667,1.919054
7743940590,105354.060000,203184.371923,1.928586


In [466]:
val_7[val_7.val < 1]

,cnt1,cnt2,val
inn,,,
5003021368,301,298.0,0.990033
5030032182,66,65.0,0.984848
5074019220,103,102.0,0.990291
5074045703,180,178.0,0.988889
7701377657,60,59.0,0.983333
...,...,...,...
7736076587,199,198.0,0.994975
7743020747,33,30.0,0.909091
7743021300,55,54.0,0.981818


In [467]:
val_2_6[val_2_6.dezil > 1.9]

,qt01,qt09,dezil
inn,,,
5003021495,86119.774714,185506.984714,2.154058
5003021640,99426.023590,190845.823205,1.919476
5003096290,93784.508021,192694.359271,2.054650
5030032182,90973.248810,184630.170476,2.029500
5051005670,99494.592471,191129.200287,1.921001
...,...,...,...
7743935328,86999.878519,175442.962222,2.016589
7743939315,84414.840000,161996.636667,1.919054
7743940590,105354.060000,203184.371923,1.928586


In [399]:
t01

,t01
inn,
5003021368,98080.876667
5003021495,91560.225000
5003021640,107809.704000
5003096290,97221.432667
5030032182,100741.000000
...,...
7743937357,101273.536000
7743939315,86853.176000
7743940590,109919.825000


In [400]:
t09

,t09
inn,
5003021368,157830.923333
5003021495,163889.401000
5003021640,167815.105000
5003096290,163329.844000
5030032182,171630.670000
...,...
7743937357,150070.824000
7743939315,153103.166000
7743940590,175252.352000


In [501]:
print_df(val_2_6, 'dezil_teach')

In [502]:
print_df(val_15, 'val_15')
print_df(teach_perc_out, 'val_15_ppl')

In [76]:
def summ_isp(row, months):
    sum = 0
    for month in months:
        if row['job_' + month] in isp:
            sum += row['sum_' + month]
    if pd.isna(row['job_' + months[-1]]):
        return np.nan
    return sum

In [97]:
prep = create_full_res(months)

In [77]:
def sum_fot(row, months):
    sum = 0
    for month in months:
        if pd.isna(row['sum_' + month]) == False:
            sum += row['sum_' + month]
    return sum

In [98]:
prep['summ'] = prep.apply(lambda row: summ_isp(row, months), axis=1)
val_1_3_ppl = prep[(prep.summ.isna() == False) & (prep.summ > 0)]
val_1_3_ppl_2 = prep.copy()
val_1_3 = prep[['inn'] + ['sum_' + month for month in months]].groupby('inn').sum().reset_index()
val_1_3['sum'] = val_1_3.apply(lambda row: sum_fot(row, months), axis=1)
prep = prep[['inn', 'summ']].groupby('inn').sum().reset_index()
val_1_3 = pd.merge(prep[['inn', 'summ']], val_1_3[['inn', 'sum']], how='left', on='inn')
val_1_3['ans'] = val_1_3['summ'] / val_1_3['sum']
val_1_3

,inn,summ,sum,ans
0,5.003021e+09,2.351476e+08,2.747557e+08,0.855843
1,5.003021e+09,2.803509e+08,3.287882e+08,0.852679
2,5.003022e+09,2.110672e+08,2.462824e+08,0.857013
3,5.003096e+09,1.221361e+08,1.558069e+08,0.783894
4,5.030032e+09,6.969081e+07,8.427697e+07,0.826926
...,...,...,...,...
592,7.743941e+09,8.524072e+07,1.063156e+08,0.801770
593,9.705102e+09,0.000000e+00,1.461994e+08,0.000000
594,9.715207e+09,1.132425e+08,1.497550e+08,0.756185
595,9.715218e+09,5.513450e+07,6.823824e+07,0.807971


In [101]:
val_1_3[val_1_3.inn == 7725031305]

,inn,summ,sum,ans
315,7.725031e+09,40727882.06,67143122.08,0.606583


In [99]:
print_df(val_1_3_ppl_2, 'ppl_2_va_1_9')

In [66]:
val_1_3[val_1_3.inn == 7720258550]

,inn,summ,sum,ans
221,7.720259e+09,1688689.6,2130513.06,0.792621


In [67]:
val_1_3[val_1_3.ans < 0.75]

,inn,summ,sum,ans
12,7.701032e+09,9834836.59,1.443857e+07,0.681150
13,7.701051e+09,33369028.04,5.181362e+07,0.644020
14,7.701114e+09,9305905.42,1.411069e+07,0.659493
16,7.701376e+09,11992089.45,1.671504e+07,0.717443
19,7.701407e+09,13441558.86,2.011727e+07,0.668160
...,...,...,...,...
580,7.743086e+09,45383049.94,7.090885e+07,0.640020
581,7.743086e+09,65806734.74,9.981817e+07,0.659266
590,7.743937e+09,50073440.48,8.381987e+07,0.597393
593,9.705102e+09,0.00,1.452827e+08,0.000000


In [21]:
temp = create_full_res(months)

In [22]:
temp['szp_vosp'] = temp.apply(lambda row: szp_vosp(row, months), axis=1)
temp['szp_teach'] = temp.apply(lambda row: szp_teach(row, months), axis=1)

In [52]:
temp[temp.snils == '001-690-637 18']

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar,szp_vosp,szp_teach
4050,7.731174e+09,001-690-637 18,76546.94,1.0,Воспитатель,76546.94,1.0,Воспитатель,76546.94,1.0,Воспитатель,76546.94,NaN
174019,7.731230e+09,001-690-637 18,NaN,NaN,NaN,NaN,NaN,NaN,76546.94,1.0,Воспитатель,76546.94,NaN


In [61]:
def give_grp(row):
    if pd.isna(row['szp_teach']):
        return 0
    if row['szp_teach'] < 100000:
        return 1
    if row['szp_teach'] < 130000:
        return 2
    if row['szp_teach'] < 150000:
        return 3
    return 4

In [72]:
def give_grp2(row):
    if row['szp_teach'] < 80000:
        return 1
    if row['szp_teach'] > 150000:
        return 2
    return 0

In [64]:
temp['grp'] = temp.apply(lambda row: give_grp(row), axis=1)

In [73]:
temp['grp2'] = temp.apply(lambda row: give_grp2(row), axis=1)

In [75]:
temp[temp.grp2 == 2]

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar,szp_vosp,szp_teach,grp,grp2
3,7.736615e+09,001-010-312 25,146547.14,1.0,Учитель,164214.30,1.0,Учитель,160599.30,1.0,Учитель,NaN,157120.246667,4,2
4,7.723170e+09,001-011-996 67,167905.34,1.0,Учитель,161441.63,1.0,Учитель,166752.15,1.0,Учитель,NaN,165366.373333,4,2
111,7.702833e+09,001-131-908 67,160320.62,1.0,Учитель,152088.02,1.0,Учитель,175707.24,1.0,Учитель,NaN,162705.293333,4,2
163,7.721151e+09,001-152-590 79,150251.40,1.0,Учитель,147882.00,1.0,Учитель,159644.78,1.0,Учитель,NaN,152592.726667,4,2
187,7.736051e+09,001-163-926 92,150559.81,0.9,Учитель,159502.90,0.9,Учитель,166829.14,0.9,Учитель,NaN,158963.950000,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175763,7.731243e+09,151-834-736 67,NaN,NaN,NaN,NaN,NaN,NaN,153568.01,1.0,Учитель,NaN,153568.010000,4,2
175766,7.731243e+09,151-908-182 62,NaN,NaN,NaN,NaN,NaN,NaN,152761.14,1.0,Учитель,NaN,152761.140000,4,2
175840,7.731243e+09,155-635-434 76,NaN,NaN,NaN,NaN,NaN,NaN,182367.50,1.0,Учитель,NaN,182367.500000,4,2
176199,7.723170e+09,175-284-645 98,NaN,NaN,NaN,NaN,NaN,NaN,221638.48,1.0,Учитель,NaN,221638.480000,4,2


In [69]:
temp[temp.grp == 0]

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar,szp_vosp,szp_teach,grp
0,7.720259e+09,001-005-586 64,124968.88,1.0,Заведующий хозяйством,140968.88,1.0,Заведующий хозяйством,60434.74,1.0,Заведующий хозяйством,NaN,NaN,0
5,7.727124e+09,001-017-453 65,87135.84,1.0,Секретарь руководителя,90135.84,1.0,Документовед,86627.92,1.0,Секретарь руководителя,NaN,NaN,0
7,7.701051e+09,001-020-587 55,81028.79,1.0,Старший методист,112852.50,1.0,Старший методист,122370.00,1.0,Старший методист,NaN,NaN,0
8,7.729776e+09,001-020-711 41,60020.00,1.0,Воспитатель,65993.00,1.0,Воспитатель,68000.00,1.0,Воспитатель,64671.0,NaN,0
9,7.723356e+09,001-021-199 51,61918.39,1.0,Преподаватель,66502.39,1.0,Преподаватель,59040.96,1.0,Преподаватель,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176757,7.734409e+09,215-089-180 55,NaN,NaN,NaN,NaN,NaN,NaN,25039.34,1.0,Специалист,NaN,NaN,0
176758,7.736615e+09,215-093-175 39,NaN,NaN,NaN,NaN,NaN,NaN,38178.00,1.0,Уборщик территории,NaN,NaN,0
176759,7.743021e+09,215-094-406 39,NaN,NaN,NaN,NaN,NaN,NaN,10015.73,1.0,Рабочий по комплексному обслуживанию и ремонту...,NaN,NaN,0
176760,7.736051e+09,215-120-882 18,NaN,NaN,NaN,NaN,NaN,NaN,13916.40,1.0,Рабочий по комплексному обслуживанию и ремонту...,NaN,NaN,0


In [76]:
print_df(temp, 'Учителя по группам')

In [71]:
temp[['grp', 'szp_teach']].groupby('grp').mean()

,szp_teach
grp,
0,NaN
1,85445.003737
2,115042.855962
3,139043.918370
4,170377.186003


In [77]:
temp[['grp2', 'szp_teach']].groupby('grp2').mean()

,szp_teach
grp2,
0,114677.789469
1,66874.387804
2,170377.186003


In [79]:
temp[['grp'] + ['stv_' + month for month in months]].groupby('grp').mean()

,stv_feb,stv_jan,stv_mar
grp,,,
0,0.995939,0.996425,0.995493
1,1.050247,1.050831,1.049498
2,1.059321,1.059312,1.059627
3,1.064928,1.064571,1.065099
4,1.065423,1.065588,1.065466


In [70]:
temp[temp.inn == 7720258550]

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar,szp_vosp,szp_teach
56260,7.720259e+09,037-829-622 90,309703.65,1.000000,Заместитель директора,132119.81,1.00,Заместитель директора,NaN,NaN,NaN,NaN,NaN
127740,7.720259e+09,139-912-912 93,87813.54,1.170000,Учитель,84213.54,1.17,Учитель,84113.54,1.170000,Учитель,NaN,85380.206667
137448,7.720259e+09,149-672-801 06,55819.15,1.390000,Учитель,84133.14,1.39,Учитель,84133.14,1.390000,Учитель,NaN,74695.143333
141545,7.720259e+09,154-173-576 64,71965.84,1.111111,Учитель,68754.74,1.22,Учитель,84145.90,1.111111,Учитель,NaN,74955.493333
145612,7.720259e+09,159-934-072 11,70922.47,1.050000,Учитель,84146.62,1.00,Учитель,78535.34,1.050000,Учитель,NaN,77868.143333
146853,7.720259e+09,162-011-849 20,84131.56,1.110000,Учитель,98727.59,1.11,Учитель,66104.80,1.110000,Учитель,NaN,82987.983333
147617,7.720259e+09,163-099-703 82,84131.84,1.670000,Учитель,80352.86,1.67,Учитель,84131.84,1.670000,Учитель,NaN,82872.180000
153843,7.720259e+09,173-181-439 62,84144.27,1.050000,Учитель,84133.94,1.05,Учитель,84133.94,1.050000,Учитель,NaN,84137.383333


In [71]:
print_df(temp[temp.inn == 7720258550], 'sch_check')

In [44]:
val_1_3[val_1_3.ans < 0.75]

,inn,summ,sum,ans
12,7701032028,9834836.59,1.466105e+07,0.670814
13,7701050524,33187221.52,5.157008e+07,0.643536
14,7701113894,9211198.82,1.404044e+07,0.656048
16,7701375995,11992089.45,1.670279e+07,0.717969
19,7701406844,13326169.14,1.996873e+07,0.667352
...,...,...,...,...
580,7743085720,45383049.94,7.097123e+07,0.639457
581,7743085744,65806734.74,1.000288e+08,0.657878
590,7743937357,49966031.78,8.402610e+07,0.594649
593,9705101759,0.00,1.465956e+08,0.000000


In [100]:
print_df(val_1_3, 'val_1_3')
print_df(val_1_3_ppl, 'val_1_3_ppl')

In [75]:
len(val_1_3[val_1_3.ans < 0.75])

133

In [475]:
res[res.snils == '016-412-327 06']

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar,szp,szp_ped,szp_teach
15586,7712022159,016-412-327 06,139859.09,1.05,Учитель,129341.55,1.05,Учитель,129431.09,1.05,Учитель,132877.243333,132877.243333,132877.243333


In [79]:
out = res[['inn', 'szp_ped']].groupby('inn').count().rename(columns={'szp_ped': 'Количество педов'})
out

,Количество педов
inn,
5003021368,433
5003021495,447
5003021640,371
5003096290,285
5030032182,126
...,...
7743939315,42
7743940590,141
9705101759,0


In [479]:
res = load_data('mar')

In [480]:
res

,inn,snils,sum_mar,stv_mar,job_mar
1,7718309983,001-005-786 70,85809.12,1.0,Учитель
3,7736614860,001-010-312 25,160599.30,1.0,Учитель
4,7723169690,001-011-996 67,166752.15,1.0,Учитель
5,7727123537,001-017-453 65,86627.92,1.0,Секретарь руководителя
6,7733088746,001-017-484 72,86787.10,1.0,Учитель
...,...,...,...,...,...
165377,7743024406,214-822-829 56,54500.00,1.0,Уборщик служебных и производственных помещений
165378,7714890087,214-860-989 83,71680.00,1.0,Рабочий по комплексному обслуживанию и ремонту...
165380,7709944805,214-887-443 92,54536.00,1.0,Уборщик служебных и производственных помещений
165381,7734247614,214-988-963 20,65000.00,1.0,Уборщик территории


In [508]:
res[res.snils == '001-697-876 59']

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar,szp,szp_ped,szp_teach
2652,7736110982,001-697-876 59,246783.15,1.0,Художественный руководитель,248468.08,1.0,Художественный руководитель,256222.11,1.0,Художественный руководитель,250491.113333,NaN,NaN


In [506]:
#меньше 84100 учителя март
min_teach = res[res.job_mar == 'Учитель'][['inn', 'snils', 'sum_mar']].groupby('snils').max().reset_index()
print_df(min_teach[min_teach.sum_mar < 84100], 'min_teach')

In [507]:
#меньше 54500 не педы
min_nped = res[res.job_mar.isin(ped) == False][['inn', 'snils', 'sum_mar']].groupby('snils').max().reset_index()
print_df(min_nped[min_nped.sum_mar < 54500], 'min_nped')

In [92]:
out = df[df.job.isin(['Учитель'])].drop_duplicates(['inn', 'snils'])[['inn', 'snils']].groupby('inn').count().rename(columns={'snils':'Количество педов'})

In [94]:
out
wb = Workbook()
ws = wb.active
for  i in df_to_row(out, header=True):
    ws.append(i)
wb.save('teach.xlsx')

In [ ]:
school_sum = sample[['inn', 'sum']].groupby('inn').sum()

In [ ]:
school_sum

In [ ]:
s = sums.groupby(['snils']).count().reset_index()
wb = Workbook()
ws = wb.active
for  i in df_to_row(s, header=True):
    ws.append(i)
wb.save('test.xlsx')